# Tutorial 04: Agents in LangChain

In this tutorial, we'll explore Agents in LangChain, which are autonomous entities capable of using tools and making decisions to accomplish tasks.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import load_tools, initialize_agent, AgentType,create_react_agent, AgentExecutor
from langchain.tools import Tool,DuckDuckGoSearchRun
# from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
# from langchain_community.tools.playwright.utils import create_sync_playwright_browser
# from langgraph.prebuilt import create_react_agent

# Load environment variables
load_dotenv()
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

# Initialize Groq LLM
llm = ChatGroq(
        model_name="qwen-2.5-32b",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

## 1. Understanding Agent Architecture

Agents in LangChain combine an LLM with a set of tools to perform tasks. The LLM decides which tools to use and how to interpret their outputs.

## 2. Exploring Different Types of Agents

### Zero-shot React Agent

In [2]:
search = DuckDuckGoSearchRun()

def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = load_tools(["wikipedia", "llm-math"], llm=llm)

zero_shot_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

zero_shot_agent.invoke(
    "What is the square root of the year Pythagoras was born?"
)

/tmp/ipykernel_30669/203328799.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  zero_shot_agent = initialize_agent(




> Entering new AgentExecutor chain...
I need to find out the year Pythagoras was born first, and then calculate the square root of that year.
Action: wikipedia
Action Input: Pythagoras
Observation: Page: Pythagoras
Summary: Pythagoras of Samos (Ancient Greek: Πυθαγόρας; c. 570 – c. 495 BC) was an ancient Ionian Greek philosopher, polymath, and the eponymous founder of Pythagoreanism. His political and religious teachings were well known in Magna Graecia and influenced the philosophies of Plato, Aristotle, and, through them, Western philosophy. Modern scholars disagree regarding Pythagoras's education and influences, but most agree that he travelled to Croton in southern Italy around 530 BC, where he founded a school in which initiates were allegedly sworn to secrecy and lived a communal, ascetic lifestyle.
In antiquity, Pythagoras was credited with mathematical and scientific discoveries, such as the Pythagorean theorem, Pythagorean tuning, the five regular solids, the theory of prop

{'input': 'What is the square root of the year Pythagoras was born?',
 'output': 'The square root of the approximate year Pythagoras was born (570 BC) is approximately 23.87.'}

In [3]:
#Create Web Search Tool
tools_web_search =[
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for searching the internet for current information"
    )
]

### Conversational Agent

In [4]:
conversational_agent = initialize_agent(
    tools+tools_web_search,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

conversational_agent.invoke({"input":"Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.","chat_history":""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: theory of relativity
Observation: Page: Theory of relativity
Summary: The theory of relativity usually encompasses two interrelated physics theories by Albert Einstein: special relativity and general relativity, proposed and published in 1905 and 1915, respectively. Special relativity applies to all physical phenomena in the absence of gravity. General relativity explains the law of gravitation and its relation to the forces of nature. It applies to the cosmological and astrophysical realm, including astronomy.
The theory transformed theoretical physics and astronomy during the 20th century, superseding a 200-year-old theory of mechanics created primarily by Isaac Newton. It introduced concepts including 4-dimensional spacetime as a unified entity of space and time, relativity of simultaneity, kinematic and gravitational time dilation, and length contraction. In the field of ph

{'input': "Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.",
 'chat_history': '',
 'output': 'The theory of relativity is a cornerstone of modern physics, primarily developed by Albert Einstein. It includes two main parts: special relativity and general relativity. Special relativity, published in 1905, deals with the physics of objects moving at constant velocity, especially those approaching the speed of light. General relativity, published in 1915, extends these ideas to include gravity and acceleration, describing gravity as the curvature of spacetime caused by mass and energy.\n\nWould you like to explore a specific aspect of the theory of relativity or a particular math problem related to it?'}

### STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION Agent

In [5]:
plan_and_execute_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

plan_and_execute_agent.run(
    "Find out who invented the telephone and calculate the number of years between that invention and the first moon landing."
)

/tmp/ipykernel_30669/2976134799.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  plan_and_execute_agent.run(




> Entering new AgentExecutor chain...


Thought: I need to find out who invented the telephone and the year it was invented. I will use the wikipedia tool for this.

Action:
```
{
  "action": "wikipedia",
  "action_input": "invention of the telephone"
}
```

Observation: Page: Invention of the telephone
Summary: The invention of the telephone was the culmination of work done by more than one individual, and led to an array of lawsuits relating to the patent claims of several individuals and numerous companies. Notable people included in this were Antonio Meucci, Philipp Reis, Elisha Gray and Alexander Graham Bell.

Page: History of the telephone
Summary: This history of the telephone chronicles the development of the electrical telephone, and includes a brief overview of its predecessors.  The first telephone patent was granted to Alexander Graham Bell in 1876.

Page: Timeline of the telephone
Summary: This timeline of the telephone covers landline, radio, and cellular telephony technologies and provides many important dates

'The telephone was invented by Alexander Graham Bell in 1876. The first moon landing occurred in 1969. The number of years between the invention of the telephone and the first moon landing is 93 years.'

## 3. Creating Custom Tools for Agents

In [6]:
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

word_length_tool = Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)

custom_tools = [word_length_tool] + tools

custom_agent = initialize_agent(
    custom_tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

custom_agent.run(
    "What's the length of the word 'supercalifragilisticexpialidocious' and its square root?"
)



> Entering new AgentExecutor chain...


Thought: I need to find the length of the word 'supercalifragilisticexpialidocious' and calculate the square root of that length. I will use the WordLength tool to get the length of the word and the Calculator tool to find the square root.

Action:
```
{
  "action": "WordLength",
  "action_input": "supercalifragilisticexpialidocious"
}
```

Observation: 34
Thought:I have the length of the word 'supercalifragilisticexpialidocious', which is 34. Now, I need to calculate the square root of 34 using the Calculator tool.

Action:
```
{
  "action": "Calculator",
  "action_input": "sqrt(34)"
}
```


Observation: Answer: 5.830951894845301
Thought:I know what to respond

Action:
```
{
  "action": "Final Answer",
  "action_input": "The length of the word 'supercalifragilisticexpialidocious' is 34 and the square root of 34 is approximately 5.83."
}
```

> Finished chain.


"The length of the word 'supercalifragilisticexpialidocious' is 34 and the square root of 34 is approximately 5.83."

## 4. Implementing a Multi-tool Agent for Task Solving

In [7]:
# Combine all tools
all_tools = tools + tools_web_search + [word_length_tool]

multi_tool_agent = initialize_agent(
    all_tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

multi_tool_agent.run(
    """Please perform the following tasks:
    1. Find the current temperature in New York City.
    2. Calculate the square root of that temperature.
    3. Find a famous quote about weather and tell me how many characters it has.
    Provide a summary of your findings.
    """
)



> Entering new AgentExecutor chain...


Thought: I need to find the current temperature in New York City. I will use the DuckDuckGo Search tool for this.
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "current temperature in New York City"
}
```

Observation: Current conditions at New York City, Central Park (KNYC) Lat: 40.78°N Lon: 73.97°W Elev: 154ft. Current conditions at New York City, Central Park (KNYC) Lat: 40.78°N Lon: 73.97°W Elev: 154ft.NOAA National Weather Service New York City Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the New York City area. New York, New York - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data. New York 7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV
Thought:I need

"The current temperature in New York City is 43°F. The square root of this temperature is approximately 6.56. A famous quote about weather is 'Weather is a great metaphor for life - sometimes it's good, sometimes it's bad, and there's nothing much you can do about it but carry an umbrella or choose to dance in the rain!' - Terri Guillemets, which has 197 characters."

## Conclusion

In this tutorial, we've explored Agents in LangChain, including different types of agents, creating custom tools, and implementing a multi-tool agent for complex task solving. Agents provide a powerful way to create autonomous systems that can leverage various tools and make decisions to accomplish tasks.